In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Feb  6 16:31:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
c_path = "drive/My Drive/praka/paper/full_corpus.csv"

In [4]:
corpus = pd.read_csv(c_path)

In [5]:
corpus

,Unnamed: 0,label,text,corp,fold
0,0,1,"да, это раздражает и громоздко отделяет ваш му...",argmicro,3
1,1,1,три разных мешка для мусора воняют на кухне и ...,argmicro,0
2,2,1,но все же германия производит слишком много му...,argmicro,0
3,3,1,"и слишком много ресурсов теряется, когда то, ч...",argmicro,0
4,5,1,в фридрихсхайн или нойкленн в наши дни трудно ...,argmicro,0
...,...,...,...,...,...
32769,24998,0,Плей-офф уберет легкий график и сделает чемпио...,ukp,0
32770,24999,1,"Например, сикхские мальчики, ортодоксальные ев...",ukp,0
32771,25000,0,"«У нас было много родительского вклада, - сказ...",ukp,4
32772,25001,1,Школьная форма может быть уродливой и / или не...,ukp,4


## Micro prep

In [6]:
df_micro = df_list[0]
df_pers = df_list[1]

NameError: ignored

In [ ]:
corpora_m = ['argmicro' for i in range(0, df_micro.shape[0])]
corpora_p = ['persessay' for i in range(0, df_pers.shape[0])]

In [ ]:
df_micro['corp'] = np.array(corpora_m)
df_pers['corp'] = np.array(corpora_p)

In [ ]:
df_pers.value_counts('label')

label
pro     4599
neut    1229
mcl      746
opp      703
dtype: int64

In [ ]:
df = pd.concat([df_micro, df_pers])
df.drop('Unnamed: 0', axis=1, inplace=True)
df = df[df.label != 'mcl']#pd.concat([df[df.label == "opp"], df[df.label == "pro"], df[df.label == "neut"]])
labels = np.array([1 if t == 'pro' or t == 'opp' else 0 for t in df.label])
df['label'] = labels

df

,label,text,file,corp
0,1,"да, это раздражает и громоздко отделяет ваш му...",micro_b001.xml,argmicro
1,1,три разных мешка для мусора воняют на кухне и ...,micro_b001.xml,argmicro
2,1,но все же германия производит слишком много му...,micro_b001.xml,argmicro
3,1,"и слишком много ресурсов теряется, когда то, ч...",micro_b001.xml,argmicro
5,1,в фридрихсхайн или нойкленн в наши дни трудно ...,micro_b002.xml,argmicro
...,...,...,...,...
7271,1,"Если вы их слишком быстро, то первое подтвержд...",micro_e99.xml,persessay
7272,1,"Это превращает вас к человеку, который имеет н...",micro_e99.xml,persessay
7273,1,Даже если мы будем игнорировать этот результат...,micro_e99.xml,persessay
7275,1,"Получить больше навыков по изучению, имеющие б...",micro_e99.xml,persessay


In [ ]:
df.value_counts('label')

label
1    6538
0    1233
dtype: int64

## Union

In [ ]:
data = pd.concat([df, df_ukp])
data.label.value_counts()

1    17561
0    15213
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
print(skf)

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


In [ ]:
!pip freeze | grep scikit-learn

scikit-learn==0.22.2.post1


In [ ]:
skf.split(data, data.label)

<generator object _BaseKFold.split at 0x7fd893bbfed0>

In [ ]:
X = [1,2,3,4,5,6,7,8,9,10]
y = [1,2,1,2,1,2,1,2,1,2]

In [ ]:
i = 0
folds = np.zeros(data.shape[0], dtype=int)
for train_index, test_index in skf.split(data, data.label):
  print(len(test_index))
  for j in test_index:
    folds[j] = i
  i = i+1

6555
6555
6555
6555
6554


In [ ]:
len([1 for i in folds if i == 4])

6554

In [ ]:
data['fold'] = folds

In [ ]:
data.drop('file', axis=1, inplace=True)
data

,label,text,corp,fold
0,1,"да, это раздражает и громоздко отделяет ваш му...",argmicro,3
1,1,три разных мешка для мусора воняют на кухне и ...,argmicro,0
2,1,но все же германия производит слишком много му...,argmicro,0
3,1,"и слишком много ресурсов теряется, когда то, ч...",argmicro,0
5,1,в фридрихсхайн или нойкленн в наши дни трудно ...,argmicro,0
...,...,...,...,...
24998,0,Плей-офф уберет легкий график и сделает чемпио...,ukp,0
24999,1,"Например, сикхские мальчики, ортодоксальные ев...",ukp,0
25000,0,"«У нас было много родительского вклада, - сказ...",ukp,4
25001,1,Школьная форма может быть уродливой и / или не...,ukp,4


In [ ]:
data.to_csv('drive/My Drive/praka/paper/full_corpus.csv')

In [7]:
data = pd.read_csv('drive/My Drive/praka/paper/full_corpus.csv')

In [8]:
f1 = data[data.fold == 0]
print('Fold 0 \nLenght: {}'.format(len(f1)))
print('Arguments: {}'.format(len(f1[f1.label == 1])))
print('NoArguments: {}'.format(len(f1[f1.label == 0])))
print('ArgMicro: {}, PersEssay: {}, UKP: {}'.format(len(f1[f1.corp == 'argmicro']), len(f1[f1.corp == 'persessay']), len(f1[f1.corp == 'ukp'])))


f1 = data[data.fold == 1]
print('\nFold 1 \nLenght: {}'.format(len(f1)))
print('Arguments: {}'.format(len(f1[f1.label == 1])))
print('NoArguments: {}'.format(len(f1[f1.label == 0])))
print('ArgMicro: {}, PersEssay: {}, UKP: {}'.format(len(f1[f1.corp == 'argmicro']), len(f1[f1.corp == 'persessay']), len(f1[f1.corp == 'ukp'])))


f1 = data[data.fold == 2]
print('\nFold 2 \nLenght: {}'.format(len(f1)))
print('Arguments: {}'.format(len(f1[f1.label == 1])))
print('NoArguments: {}'.format(len(f1[f1.label == 0])))
print('ArgMicro: {}, PersEssay: {}, UKP: {}'.format(len(f1[f1.corp == 'argmicro']), len(f1[f1.corp == 'persessay']), len(f1[f1.corp == 'ukp'])))


f1 = data[data.fold == 3]
print('\nFold 3 \nLenght: {}'.format(len(f1)))
print('Arguments: {}'.format(len(f1[f1.label == 1])))
print('NoArguments: {}'.format(len(f1[f1.label == 0])))
print('ArgMicro: {}, PersEssay: {}, UKP: {}'.format(len(f1[f1.corp == 'argmicro']), len(f1[f1.corp == 'persessay']), len(f1[f1.corp == 'ukp'])))


f1 = data[data.fold == 4]
print('\nFold 4 \nLenght: {}'.format(len(f1)))
print('Arguments: {}'.format(len(f1[f1.label == 1])))
print('NoArguments: {}'.format(len(f1[f1.label == 0])))
print('ArgMicro: {}, PersEssay: {}, UKP: {}'.format(len(f1[f1.corp == 'argmicro']), len(f1[f1.corp == 'persessay']), len(f1[f1.corp == 'ukp'])))



Fold 0 
Lenght: 6555
Arguments: 3513
NoArguments: 3042
ArgMicro: 234, PersEssay: 1319, UKP: 5002

Fold 1 
Lenght: 6555
Arguments: 3512
NoArguments: 3043
ArgMicro: 236, PersEssay: 1332, UKP: 4987

Fold 2 
Lenght: 6555
Arguments: 3512
NoArguments: 3043
ArgMicro: 258, PersEssay: 1334, UKP: 4963

Fold 3 
Lenght: 6555
Arguments: 3512
NoArguments: 3043
ArgMicro: 257, PersEssay: 1299, UKP: 4999

Fold 4 
Lenght: 6554
Arguments: 3512
NoArguments: 3042
ArgMicro: 255, PersEssay: 1247, UKP: 5052


In [9]:
f1 = data[data.corp == 'argmicro']
print('ArgMicro\nArguments: {}, NoArguments: {}'.format(len(f1[f1.label == 1]), len(f1[f1.label == 0])))


f1 = data[data.corp == 'persessay']
print('\nPersEssay\nArguments: {}, NoArguments: {}'.format(len(f1[f1.label == 1]), len(f1[f1.label == 0])))


f1 = data[data.corp == 'ukp']
print('\nUKP\nArguments: {}, NoArguments: {}'.format(len(f1[f1.label == 1]), len(f1[f1.label == 0])))


f1 = data
print('\nFull\nArguments: {}, NoArguments: {}'.format(len(f1[f1.label == 1]), len(f1[f1.label == 0])))

ArgMicro
Arguments: 1236, NoArguments: 4

PersEssay
Arguments: 5302, NoArguments: 1229

UKP
Arguments: 11023, NoArguments: 13980

Full
Arguments: 17561, NoArguments: 15213


# Bert import

In [6]:
!pip install transformers
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments, TFTrainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support, roc_auc_score
import torch
from torch.nn.functional import softmax
import tensorflow
import matplotlib.pyplot as plt
from time import time
import sys
from subprocess import call
!pip install datasets
from datasets import Dataset

     |████████████████████████████████| 3.5 MB 4.1 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 6.8 MB 38.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 4.3 MB/s 
     |████████████████████████████████| 243 kB 26.1 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 133 kB 38.7 MB/s 
     |████████████████████████████████| 144 kB 48.8 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 49.4 MB/s 


In [7]:
PRE_TRAINED_MODEL_NAME = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

PreTrainedTokenizer(name_or_path='DeepPavlov/rubert-base-cased', vocab_size=119547, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [8]:
def tokenize(batch):
    '''Функция для токенизации Dataset'''
    return tokenizer(
        batch['text'],
        max_length=512,
        padding='max_length',        # 'longest', 'max_length', 'do_not_pad'
        truncation='longest_first'
    )

## Make folds

### Metrics

In [9]:
#функция записи в файл и вывода значений метрик
def write_metrics(test_predict_results,test_labels, f, f_csv):
  test_pred = [np.argmax(a) for a in test_predict_results.predictions]
  classification_dict = classification_report(test_labels, test_pred, digits=3, output_dict=True)
  print(classification_dict)
  # cm = confusion_matrix(test_labels, test_pred)
  # print(f'TN: {cm[0][0]}\nFP: {cm[0][1]}\nFN: {cm[1][0]}\nTP: {cm[1][1]}\n')
  # f.write(f'TN: {cm[0][0]}\nFP: {cm[0][1]}\nFN: {cm[1][0]}\nTP: {cm[1][1]}\n')
  # print('class opp')
  # f.write('\nclass opp\n')
  # print(f"precision = {classification_dict['opp']['precision']}")
  # f.write(f"precision = {classification_dict['opp']['precision']}\n")
  # print(f"recall = {classification_dict['opp']['recall']}")
  # f.write(f"recall = {classification_dict['opp']['recall']}\n")
  # print(f"f1-score = {classification_dict['opp']['f1-score']}\n")
  # f.write(f"f1-score = {classification_dict['opp']['f1-score']}\n")
  # print('class pro')
  # f.write('\nclass pro\n')
  # print(f"precision = {classification_dict['pro']['precision']}\n")
  # f.write(f"precision = {classification_dict['pro']['precision']}\n")
  # print(f"recall = {classification_dict['pro']['recall']}")
  # f.write(f"recall = {classification_dict['pro']['recall']}\n")
  # print(f"f1-score = {classification_dict['pro']['f1-score']}\n")
  # f.write(f"f1-score = {classification_dict['pro']['f1-score']}\n")
  # accuracy = classification_dict['accuracy']
  # print(f"accuracy = {accuracy}\n")
  # f.write(f"\naccuracy = {accuracy}\n")
  # print('macro avg')
  # f.write('\nmacro avg\n')
  # precision = classification_dict['macro avg']['precision']
  # print(f"precision = {precision}")
  # f.write(f"precision = {precision}\n")
  # recall = classification_dict['macro avg']['recall']
  # print(f"recall = {recall}")
  # f.write(f"recall = {recall}\n")
  # f1 = classification_dict['macro avg']['f1-score']
  # print(f"f1-score = {f1}")
  # f.write(f"f1-score = {f1}\n")

  # f_csv.write(f"{f1};{precision};{recall};{accuracy};")
  # f_csv.write(f"{cm[0][0]};{cm[0][1]};{cm[1][0]};{cm[1][1]};")
  # f_csv.write(f"{classification_dict['opp']['precision']};{classification_dict['opp']['recall']};{classification_dict['opp']['f1-score']};{classification_dict['pro']['precision']};{classification_dict['pro']['recall']};{classification_dict['pro']['f1-score']};")

  return accuracy, precision, recall, f1

In [10]:
def compute_metrics(pred):
    '''Функция возвращает метрики качества'''
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1.tolist(),
        'precision': precision.tolist(),
        'recall': recall.tolist()
    }

# CV loop

In [11]:
torch.manual_seed(42)
np.random.seed(42)

In [90]:
#списки для сводных данных по фолдам
accuracy = []
precision = []
recall = []
macro_f1 = []

#PRE_TRAINED_MODEL_NAME = 'DeepPavlov/rubert-base-cased'
#PRE_TRAINED_MODEL_NAME = 'drive/My Drive/am_paper_2'
PRE_TRAINED_MODEL_NAME = 'drive/My Drive/Experiment_urgent'

In [91]:
training_args = TrainingArguments(
    output_dir='./results',          # выходной каталог
    overwrite_output_dir=True,       # перезаписывать контрольные точки
    save_steps=5000,                 # количество шагов до обновления контрольной точки
    num_train_epochs=5,              # количество эпох обучения
    per_device_train_batch_size=8,  # размер батча на устройство при обучении
    per_device_eval_batch_size=8,   # размер батча при оценке
    weight_decay=0.01,               # strength of weight decay
    learning_rate=1e-5,              # скорость обучения
    logging_dir='./logs',            # каталог для логов
    evaluation_strategy='no'     # 'no', 'steps', 'epoch' (оценка в конце каждой эпохи)
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [92]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [93]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [94]:
torch.cuda.memory_cached()

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:386: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


769654784

In [95]:
def def_model():
  model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path=PRE_TRAINED_MODEL_NAME)
  model.to(device)
  #model.config
  return model

## No folds

In [ ]:
train_init_texts = list(data.text)
train_init_labels = list(data.label)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_init_texts, train_init_labels, test_size = 0.1, random_state=42,  stratify=train_init_labels)

In [ ]:
#df_val = pd.DataFrame({'label': val_labels, 'text': val_texts })
df_train = pd.DataFrame({'label': train_init_labels, 'text': train_init_texts })

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
#eval_dataset = Dataset.from_pandas(df_val)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
#eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [96]:
#Определяем, что будет возвращать функция __getitem__() для Dataset
#train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

#eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

trainer = Trainer(
  model = def_model(),
  args=training_args,
  compute_metrics=compute_metrics
)
#trainer.model=def_model()#загрузка модели

print("trainer.model.bert.embeddings")
print(trainer.model.bert.embeddings.word_embeddings.weight)
print("trainer.model.classifier.weight")
print(trainer.model.classifier.weight)

#trainer.train_dataset = train_dataset#определение тренировочного и валидационного наборов данных
#trainer.eval_dataset = eval_dataset

loading configuration file drive/My Drive/Experiment_urgent/config.json
Model config BertConfig {
  "_name_or_path": "DeepPavlov/rubert-base-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 

trainer.model.bert.embeddings
Parameter containing:
tensor([[ 0.0041,  0.0137, -0.0080,  ...,  0.0170,  0.0175,  0.0226],
        [ 0.0010,  0.0184,  0.0111,  ..., -0.0029, -0.0008,  0.0019],
        [ 0.0025,  0.0105,  0.0073,  ..., -0.0028, -0.0003, -0.0066],
        ...,
        [ 0.0035,  0.0057,  0.0148,  ..., -0.0182, -0.0150, -0.0011],
        [-0.0337,  0.0007, -0.0863,  ..., -0.0015, -0.0248, -0.0092],
        [ 0.0178,  0.0073, -0.0294,  ...,  0.0108,  0.0266,  0.0045]],
       device='cuda:0', requires_grad=True)
trainer.model.classifier.weight
Parameter containing:
tensor([[-2.2454e-02,  1.8472e-02,  2.9527e-02,  ..., -1.8087e-03,
          1.5084e-02, -2.4941e-06],
        [ 1.3157e-03, -1.0866e-03,  7.5381e-03,  ...,  4.4433e-03,
         -1.8473e-02, -2.8115e-02]], device='cuda:0', requires_grad=True)


In [ ]:
trainer.train()

In [ ]:
#trainer.save_model(output_dir='drive/My Drive/am_paper_2')

Saving model checkpoint to drive/My Drive/am_paper_2
Configuration saved in drive/My Drive/am_paper_2/config.json
Model weights saved in drive/My Drive/am_paper_2/pytorch_model.bin


In [ ]:
len(train_init_labels)

## folds

In [ ]:
data = corpus#[corpus.corp == 'argmicro']
data_test = data[data.fold == 4]
data_train = data[data.fold != 4]
data.value_counts('label')

label
1    17561
0    15213
dtype: int64

In [ ]:
corp = 'full'
for n in range(1, 5):

    f = open('drive/My Drive/praka/paper/metrics/'+corp+'_fold_'+str(n)+'.txt', 'w')
    f_csv = open('drive/My Drive/praka/paper/metrics/'+corp+'_fold'+str(n)+'_metr.txt','w')

    train_texts, test_texts = list(data[data.fold != n].text), list(data[data.fold == n].text)#разделение на тестовую и тренировочную части
    train_labels, test_labels = list(data[data.fold != n].label), list(data[data.fold == n].label)

    print(f'\nlen train_init: {len(train_texts)}')
    print(train_texts[:5])

    print(f'len test: {len(test_texts)}')
    print(test_texts[:5])
  
    #формирование DataFrame
    df_train = pd.DataFrame({'label': train_labels, 'text': train_texts })
    df_test = pd.DataFrame({'label': test_labels, 'text': test_texts })


    print("\nСоотношение классов")
    print(df_train.label.value_counts(normalize=True))

    #формирование Dataset
    train_dataset = Dataset.from_pandas(df_train)
    test_dataset = Dataset.from_pandas(df_test)    

    #токенизация
    train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
    test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

    #Определяем, что будет возвращать функция __getitem__() для Dataset
    train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    trainer = Trainer(
      model = def_model(),
      args=training_args,
      compute_metrics=compute_metrics
    )
    #trainer.model=def_model()#загрузка модели

    print("trainer.model.bert.embeddings")
    print(trainer.model.bert.embeddings.word_embeddings.weight)
    print("trainer.model.classifier.weight")
    print(trainer.model.classifier.weight)

    trainer.train_dataset = train_dataset #определение тренировочного наборов данных

    #обучение
    start_time = time()
    trainer.train()
    end_time = time() - start_time
    print(n)
    f.write(f'\n{n}\n')
    print(f'\nTime train: {end_time:.1f} sec\n')
    f.write(f'\nTime train: {end_time:.1f} sec\n') 

    #оценка модели на тестовом наборе данных
    start_time = time()
    test_predict_results = trainer.predict(test_dataset)
    print(f'\nTime to calculate metrics: {time() - start_time:.1f} sec\n')    
    f.write(f'\nTime to calculate metrics: {time() - start_time:.1f} sec\n') 
    #acc, prec, rec, f1 = write_metrics(test_predict_results,test_labels,f, f_csv)
    
    test_pred = [np.argmax(a) for a in test_predict_results.predictions]
    df_preds = pd.DataFrame(np.array(test_pred), columns=['pred_label'])
    df_preds.to_json('drive/My Drive/praka/paper/metrics/'+corp+'_fold_'+str(n)+'.json', force_ascii=False, orient='records')
    
    f.close()
    f_csv.close()



len train_init: 26219
['да, это раздражает и громоздко отделяет ваш мусор должным образом все время.', 'три разных мешка для мусора воняют на кухне и их нужно сортировать в разные корзины.', 'но все же германия производит слишком много мусора', 'и слишком много ресурсов теряется, когда то, что на самом деле должно быть разделено и переработано, сжигается.', 'в фридрихсхайн или нойкленн в наши дни трудно двигаться, не просматривая почву для собак.']
len test: 6555
['во-первых, неприкосновенное человеческое достоинство закреплено в нашей конституции,', 'владельцы собак могут подумать, что собачья грязь - это всего лишь безобидная секреция животного, как и любая другая.', 'тем не менее, они недооценивают его влияние на дорожное движение.', 'но это не так просто.', 'тем более что для адекватной прибыли арендная ставка по сравнению с базовой арендой не должна повышаться для каждой новой аренды.']

Соотношение классов
1    0.535833
0    0.464167
Name: label, dtype: float64


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
***** Running training *****
  Num examples = 26219
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16390


...

trainer.model.bert.embeddings
Parameter containing:
tensor([[ 0.0041,  0.0138, -0.0080,  ...,  0.0170,  0.0175,  0.0226],
        [ 0.0010,  0.0184,  0.0111,  ..., -0.0029, -0.0008,  0.0019],
        [ 0.0025,  0.0105,  0.0073,  ..., -0.0028, -0.0003, -0.0066],
        ...,
        [ 0.0035,  0.0057,  0.0148,  ..., -0.0182, -0.0150, -0.0011],
        [-0.0342,  0.0004, -0.0860,  ..., -0.0019, -0.0255, -0.0084],
        [ 0.0178,  0.0073, -0.0294,  ...,  0.0108,  0.0266,  0.0045]],
       device='cuda:0', requires_grad=True)
trainer.model.classifier.weight
Parameter containing:
tensor([[-0.0602, -0.0056, -0.0061,  ..., -0.0094, -0.0017, -0.0018],
        [-0.0100,  0.0176, -0.0264,  ...,  0.0152, -0.0154,  0.0013]],
       device='cuda:0', requires_grad=True)


Step,Training Loss
500,0.577200
1000,0.498700
1500,0.505100
2000,0.479400
2500,0.497000
3000,0.464100
3500,0.408600
4000,0.391300
4500,0.377900
5000,0.385600


Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-10000
Configuration saved in ./results/checkpoint-10000/config.json
Model weights saved in ./results/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-15000
Configuration saved in ./results/checkpoint-15000/config.json
Model weights saved in ./results/checkpoint-15000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 6554
  Batch size = 8


4

Time train: 7780.4 sec




Time to calculate metrics: 126.4 sec



In [ ]:
print(classification_report([np.argmax(a) for a in test_predict_results.predictions], test_labels))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       234

    accuracy                           1.00       234
   macro avg       1.00      1.00      1.00       234
weighted avg       1.00      1.00      1.00       234



# Analysis

In [ ]:
corpus[corpus.fold == 4]

,Unnamed: 0,label,text,corp,fold
6,8,1,"конечно, сначала им нужно было быть пойманным ...",argmicro,4
7,9,1,"но как только им придется копаться в карманах,...",argmicro,4
9,12,1,тем не менее это очень доказательство отсутств...,argmicro,4
16,21,1,это должно быть ясно каждому после раскрытия э...,argmicro,4
28,35,1,"конечно, было бы здорово и просто, если после ...",argmicro,4
...,...,...,...,...,...
32753,24982,0,"С униформой также связано множество споров, пл...",ukp,4
32759,24988,1,"Они даже зашли так далеко, что сослались на по...",ukp,4
32768,24997,0,"Несмотря на то, что тесты разрабатываются штат...",ukp,4
32771,25000,0,"«У нас было много родительского вклада, - сказ...",ukp,4


In [ ]:
f_p = []

In [ ]:
for i in range(0, 5):
  df = pd.read_json('/content/drive/MyDrive/praka/paper/metrics/full_fold_'+str(i)+'.json')
  f_p.append(df)

In [ ]:
f_p[4].pred_label.count()

6555

[      pred_label
 0              0
 1              1
 2              1
 3              0
 4              1
 ...          ...
 6550           1
 6551           0
 6552           0
 6553           0
 6554           1
 
 [6555 rows x 1 columns]]

In [ ]:
pr_list = []
l_list = []
cm_list = []
for i in range(0,5):
  pr_list.append(pd.read_json('/content/drive/MyDrive/praka/paper/metrics/full_fold_'+str(i)+'.json'))
  l_list.append(corpus[corpus.fold == i])
  cm_list.append(confusion_matrix(pr_list[i].pred_label, l_list[i].label))

In [16]:
a_corpus = pd.read_json('/content/drive/MyDrive/praka/paper/news_sentences.json')

In [24]:
a

,text,word_num,df_index,sent_index,URL,topic
0,Альфа-Банк запустил регистрацию бизнеса в отде...,7,3060,0,https://www.banki.ru/news/lenta/?id=1094614,Альфа-Банк предложил в отделениях опцию регист...
1,«Мы предлагаем: давайте снизим (НДС) на продукты…,7,3064,1,https://www.banki.ru/news/lenta/?id=1094619,Миронов предложил снизить НДС на базовые проду...
2,Прогноз изменения рейтинга — «стабильный».,7,3066,2,https://www.banki.ru/news/lenta/?id=1094621,S&P повысило рейтинг Московского Кредитного Ба...
3,"«Это уже серьезная проблема, а в ближайшем буд...",7,3067,3,https://www.banki.ru/news/lenta/?id=1094622,Эксперты рассказали о получении мошенниками кр...
4,Много людей столкнется с попытками осуществлен...,7,3067,4,https://www.banki.ru/news/lenta/?id=1094622,Эксперты рассказали о получении мошенниками кр...
...,...,...,...,...,...,...
68854,"Важно убедить, в том числе и городские админис...",27,1866,71952,https://www.banki.ru/news/lenta/?id=10942580,«ВЭБ-лизинг» будет участвовать в развитии горо...
68855,При этом значительную концентрацию клиентской ...,27,1874,71953,https://www.banki.ru/news/lenta/?id=10942590,​«Эксперт РА» присвоило банку «Саратов» рейтин...
68856,"Согласно уточненным данным Росстата, в апреле ...",27,1886,71954,https://www.banki.ru/news/lenta/?id=10942609,ДИП ЦБ: инфляция замедляется в соответствии с ...
68857,"Согласно реестру обязательств Кемсоцинбанка, з...",27,1902,71955,https://www.banki.ru/news/lenta/?id=10942632,"АСВ выплатит 1,7 млрд рублей страховки вкладчи..."


In [ ]:
cm_list[4]

array([[2398,  757],
       [ 644, 2755]])

# Pred

In [97]:
b_corpus = pd.read_json('/content/drive/MyDrive/praka/paper/news_sentences.json')

In [17]:
a_corpus = pd.read_json('/content/drive/MyDrive/praka/paper/news_sentences.json')

In [18]:
a_corpus

,text,word_num,df_index,sent_index,URL,topic
0,Альфа-Банк запустил регистрацию бизнеса в отде...,7,3060,0,https://www.banki.ru/news/lenta/?id=1094614,Альфа-Банк предложил в отделениях опцию регист...
1,«Мы предлагаем: давайте снизим (НДС) на продукты…,7,3064,1,https://www.banki.ru/news/lenta/?id=1094619,Миронов предложил снизить НДС на базовые проду...
2,Прогноз изменения рейтинга — «стабильный».,7,3066,2,https://www.banki.ru/news/lenta/?id=1094621,S&P повысило рейтинг Московского Кредитного Ба...
3,"«Это уже серьезная проблема, а в ближайшем буд...",7,3067,3,https://www.banki.ru/news/lenta/?id=1094622,Эксперты рассказали о получении мошенниками кр...
4,Много людей столкнется с попытками осуществлен...,7,3067,4,https://www.banki.ru/news/lenta/?id=1094622,Эксперты рассказали о получении мошенниками кр...
...,...,...,...,...,...,...
68854,"Важно убедить, в том числе и городские админис...",27,1866,71952,https://www.banki.ru/news/lenta/?id=10942580,«ВЭБ-лизинг» будет участвовать в развитии горо...
68855,При этом значительную концентрацию клиентской ...,27,1874,71953,https://www.banki.ru/news/lenta/?id=10942590,​«Эксперт РА» присвоило банку «Саратов» рейтин...
68856,"Согласно уточненным данным Росстата, в апреле ...",27,1886,71954,https://www.banki.ru/news/lenta/?id=10942609,ДИП ЦБ: инфляция замедляется в соответствии с ...
68857,"Согласно реестру обязательств Кемсоцинбанка, з...",27,1902,71955,https://www.banki.ru/news/lenta/?id=10942632,"АСВ выплатит 1,7 млрд рублей страховки вкладчи..."


In [98]:
dataset_pr_resp = Dataset.from_pandas(b_corpus)
dataset_pr_resp = dataset_pr_resp.map(tokenize, batched=True, batch_size=len(b_corpus))
dataset_pr_resp.set_format('torch', columns=['input_ids', 'attention_mask'])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [100]:
dataset_pr_resp

Dataset({
    features: ['text', 'word_num', 'df_index', 'sent_index', 'URL', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 68859
})

In [101]:
news_predict_results = trainer.predict(dataset_pr_resp)

***** Running Prediction *****
  Num examples = 68859
  Batch size = 8


In [26]:
resp_predict_results = trainer.predict(dataset_pr_resp)

***** Running Prediction *****
  Num examples = 68859
  Batch size = 8


In [28]:
resp_predict_results

PredictionOutput(predictions=array([[ 3.9932647, -3.8170347],
       [ 3.4646482, -3.200785 ],
       [ 4.0741534, -3.8452384],
       ...,
       [ 3.5182214, -3.3113282],
       [ 3.9257698, -3.7713552],
       [-3.1296284,  2.9665148]], dtype=float32), label_ids=None, metrics={'test_runtime': 4761.5346, 'test_samples_per_second': 14.462, 'test_steps_per_second': 1.808})

In [29]:
sig_pred = sigmoid(resp_predict_results)

NameError: ignored

In [36]:
pred_np = np.array(resp_predict_results)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [103]:
np.array(tf.sigmoid(news_predict_results.predictions))[:, 1]

array([0.01080386, 0.02155128, 0.01179017, ..., 0.02799811, 0.0146933 ,
       0.9397088 ], dtype=float32)

In [104]:
b_corpus['prob_1'] = np.array(tf.sigmoid(news_predict_results.predictions))[:, 1]

In [105]:
b_corpus.sort_values("prob_1", ascending=False)

,text,word_num,df_index,sent_index,URL,topic,prob_1
24058,"«Каждой семье еще по комнате, обеспеченность ж...",12,7790,25430,https://www.banki.ru/news/lenta/?id=10944854,Власти обсуждают возможность обеспечить каждую...,0.985944
13453,"Его отмена лишит рынок защитного механизма», —...",10,6516,14239,https://www.banki.ru/news/lenta/?id=10943488,Минэнерго выступило против отмены демпфера как...,0.985915
33640,Она будет в первую очередь выгодна для сырьевы...,15,365,35481,https://www.banki.ru/news/lenta/?id=1094511,Saxo Bank увидел в мировых тенденциях риск неп...,0.985505
224,"Локации, которые можно увидеть в роликах, были...",7,3611,233,https://www.banki.ru/news/lenta/?id=10941226,"Знакомьтесь, Банки — робот Банки.ру",0.985425
40746,"«Если убрать агрессивные планы по росту, то пр...",16,5435,42864,https://www.banki.ru/news/lenta/?id=10942321,Эксперты назвали потенциальных лидеров рынка с...,0.985323
...,...,...,...,...,...,...,...
9112,"Если кто-то из чиновников что-то сказал, надо ...",9,6719,9684,https://www.banki.ru/news/lenta/?id=10943711,Решетников: МЭР не предлагало распространить к...,0.007387
17655,"Комментируя тему мисселинга , он подчеркнул: «...",11,5684,18618,https://www.banki.ru/news/lenta/?id=10942590,В Центробанке назвали количество активных инве...,0.007299
127,«Им без разницы — с кэшбэком к ним пришел ребе...,7,3349,127,https://www.banki.ru/news/lenta/?id=1094943,В РФ предложили подключить все лагеря к програ...,0.007229
6968,"«Мы смотрим на это и считаем, что речь идет о ...",9,2679,7406,https://www.banki.ru/news/lenta/?id=1094196,ФРС США не рассматривает вариант повышения ста...,0.007189


In [50]:
import tensorflow as tf

In [64]:
np.array(tf.sigmoid(pred_np))[:, 1]

array([0.02151964, 0.03913619, 0.02093371, ..., 0.0351846 , 0.02250281,
       0.95103824], dtype=float32)

In [65]:
a_corpus['prob_1'] = np.array(tf.sigmoid(pred_np))[:, 1]

In [72]:
a_corpus.sort_values("prob_1", ascending=False)

,text,word_num,df_index,sent_index,URL,topic,prob_1
36620,Сбор биометрических данных не повлияет на дост...,15,5816,38523,https://www.banki.ru/news/lenta/?id=10942728,Минцифры: сбор биометрии не повлияет на доступ...,0.979328
10707,Это обеспечит создание новых рабочих мест и ро...,10,1684,11352,https://www.banki.ru/news/lenta/?id=10942347,"В Москве поддержат предпринимателей, работающи...",0.978894
63009,"Кроме того, парки смогут выплачивать деньги за...",23,5906,65971,https://www.banki.ru/news/lenta/?id=10942827,QIWI запустила сервис по кредитованию малого и...,0.978878
55550,Биометрические данные также могут быть использ...,20,5816,58247,https://www.banki.ru/news/lenta/?id=10942728,Минцифры: сбор биометрии не повлияет на доступ...,0.978705
54155,Роботы для поставщиков сервисов и услуг могут ...,20,2231,56752,https://www.banki.ru/news/lenta/?id=10943087,​МТС начали продавать роботов,0.978695
...,...,...,...,...,...,...,...
37597,Всероссийская конференция НАПКА «Взыскание дол...,15,7518,39532,https://www.banki.ru/news/lenta/?id=10944562,​Всероссийская конференция НАПКА «Взыскание до...,0.017318
9779,График работы отделения в Казани: понедельник ...,9,7894,10400,https://www.banki.ru/news/lenta/?id=10944966,Банк ​«Фридом Финанс» открыл отделение в Казани​,0.017295
35452,Седьмой Всероссийский форум лидеров банковской...,15,3735,37336,https://www.banki.ru/news/lenta/?id=10941361,Всероссийский форум лидеров банковской розницы...,0.017267
35529,Оформить онлайн Лиц.№ 1885 Специальные предлож...,15,3873,37413,https://www.banki.ru/news/lenta/?id=10941518,Абсолют Банк запускает акцию для держателей Vi...,0.017257


In [69]:
a_corpus.to_csv('/content/drive/MyDrive/praka/full_prob_news.csv')

In [84]:
a_corpus.sort_values("prob_1", ascending=False).iloc[0:3500].to_csv('/content/drive/MyDrive/praka/3500_prob_news.csv', sep='\t')

In [89]:
resp_predict_results.predictions


array([[ 3.9932647, -3.8170347],
       [ 3.4646482, -3.200785 ],
       [ 4.0741534, -3.8452384],
       ...,
       [ 3.5182214, -3.3113282],
       [ 3.9257698, -3.7713552],
       [-3.1296284,  2.9665148]], dtype=float32)